# Parallel Trapezoidal Rule with mpi4py

In this notebook, we compute the integral of $f(x) = x^2$ over the interval $[0, 1]$ using the trapezoidal rule, parallelized with **mpi4py**.

Each MPI process computes a partial sum over a subinterval, and the results are gathered using `MPI.reduce`.

## 📦 Required Libraries

In [1]:
from mpi4py import MPI
import numpy as np

## 📐 Define the Function

In [ ]:
def f(x):
    return x**2

## ⚙️ Parallel Trapezoidal Integration with MPI

In [ ]:
# MPI setup
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Integration bounds and total number of trapezoids
a = 0.0
b = 1.0
n = 10000  # total number of intervals

# Width of each trapezoid
h = (b - a) / n

# Determine the range of indices for this process
local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h

# Compute local integral using the trapezoidal rule
x = np.linspace(local_a, local_b, local_n + 1)
local_integral = (h / 2) * (f(x[0]) + 2 * np.sum(f(x[1:-1])) + f(x[-1]))

# Reduce results to rank 0
total_integral = comm.reduce(local_integral, op=MPI.SUM, root=0)

# Rank 0 prints the result
if rank == 0:
    print(f"Estimated integral of f(x) = x^2 from 0 to 1 using parallel trapezoidal rule: {total_integral:.6f}")
    print("Exact value: 1/3 ≈ 0.333333")


## ✅ Notes
- This implementation divides the interval $[0, 1]$ evenly among all MPI processes.
- Each process computes its local contribution and sends it to rank 0 for summation.
- To run this notebook, you'll need to convert it to a script and run it using `mpirun` or `mpiexec`, for example:

```bash
mpirun -n 4 python parallel_trapezoid.py
```
